<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/BAG_PULP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача о ранце. 

Есть некоторый рюкзак с объемом $B$, есть предметы $i \in SUBJECT$ с различной стоимостью $c_i$ и весом $b_i$. Требуется найти оптимальное с точки зрения максимизации итоговой стоимости рюкзака решение, какие предметы взять.

Запишем математическую модель.

Введем переменные:

$ x_{i} = \left\{
\begin{array}{ll}
          1, & \mbox { если берем $i$ предмет в рюкзак,}\\
          0, & \mbox { в противном случае. } \\
\end{array} \right. $

Договоримся о входных данных:

$I$ - множество предметов;

$b_i$ - вес $i$ предмета;

$c_i$ - стоимость $i$ предмета.

Математическая модель:

Целевая функция:

$$\sum\limits_{i = 0}^{I} c_i x_i \to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in I $$

$$\sum\limits_{i = 0}^{I} b_i x_i \leqslant B $$

In [ ]:
!pip install pulp
!pip install cplex

In [ ]:
#import gurobipy as gp
#from gurobipy import GRB
import numpy as np
import time
from random import randrange
import pulp as plp

In [ ]:
I = 50
B = 100
c = np.random.randint(50, 70, I)
b = np.random.randint(1, 10, I)

In [ ]:
problem = plp.LpProblem(name='BagProblem', sense=plp.LpMaximize)
x  = {i : plp.LpVariable(cat=plp.LpBinary, name="x"+str(i)) for i in range(I)}

problem.addConstraint(plp.LpConstraint(
                     e=plp.lpSum(b[i] * x[i] for i in range(I)),
                     sense=plp.LpConstraintLE,
                     rhs=B ,
                     name="constraint_{1}"))

problem.setObjective(plp.lpSum(c[i] * x[i] for i in range(I)))
print(problem)

In [ ]:
solver_list = plp.list_solvers()
print(solver_list)

In [ ]:
def experiment(name, solver):
    start = time.time()
    problem.solve(solver)
    answer = plp.value(problem.objective)
    print(f"{name} solved {time.time() - start}  answer : {answer}")
    
#experiment('Gurobi', plp.get_solver('GUROBI', msg=False))
experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False))
#experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False))
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False))

In [ ]:
 for v in problem.variables():
            print(v.name, "=", v.varValue)

In [ ]:
def experiment(name, solver, timeLimit):
    times = []
    start = time.time()
    current = start
    while current - start <= timeLimit:
        problem.solve(solver)
        startIter, current = current, time.time()
        times.append(current - startIter)
    
    answer = plp.value(problem.objective)
    print(f"{name} solved {len(times) - 1} instances "
          f"in {timeLimit} seconds answer : {answer}")

    
experiment('Gurobi', plp.get_solver('GUROBI', msg=False), 120)
experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False), 120)
experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False), 120)
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False), 120)